In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('redhen_preprocessed.csv')
data.head()

,Unnamed: 0,StoryID,Excerpt,CodesApplied_Combined,ACCOUNT,ACCOUNT_Cultural,ACCOUNT_Individual,ACCOUNT_Other,COMMUNITYRECOVERY,EVENT,...,TRAUMA_Societal,VICTIMS,word_count,char_count,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean
0,7992,NI3079,Elliot Rodger Wangs roommate stabbed Wang and ...,EVENT,0,0,0,0,99,1,...,99,0,34,215.0,elliot rodger wangs roommate stabbed wang two ...,"['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'..."
1,4686,NI2689,Because the tragedy unfolded at the start of a...,RESOURCES,0,0,0,0,99,0,...,99,0,32,228.0,tragedy unfolded start holiday weekend school ...,"['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee..."
2,6130,NI2400,Gunrelated restraining orders Also passed in r...,"POLICY, POLICY- Guns, POLICY- Mental health",0,0,0,0,99,0,...,99,0,122,734.0,gunrelated restraining orders also passed resp...,"['gunrelated', 'restraining', 'orders', 'also'...","['gunrelated', 'restraining', 'orders', 'also'...","['gunrel', 'restrain', 'order', 'also', 'pass'...","['gunrelated', 'restraining', 'order', 'also',...","['gunrel', 'restrain', 'order', 'also', 'pass'..."
3,5876,NI1393,Linder said she will also remember Weiss for h...,"VICTIMS, GRIEF/LOSS",0,0,0,0,99,0,...,99,1,100,499.0,linder said also remember weiss epic shot pool...,"['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'rememb', 'weiss', ...","['linder', 'said', 'also', 'remember', 'wei', ...","['linder', 'said', 'also', 'rememb', 'weiss', ..."
4,4126,NI3259,I mean I know how this goes We all do Werent y...,"ACCOUNTABILITY, ACCOUNT- Culture/societal risk...",1,1,0,0,99,0,...,99,0,85,425.0,mean know goes werent sort expecting father on...,"['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...","['mean', 'know', 'go', 'werent', 'sort', 'expe...","['mean', 'know', 'goe', 'werent', 'sort', 'exp..."


In [4]:
data.shape

(8131, 62)

In [6]:
data.columns

Index(['Unnamed: 0', 'StoryID', 'Excerpt', 'CodesApplied_Combined', 'ACCOUNT',
       'ACCOUNT_Cultural', 'ACCOUNT_Individual', 'ACCOUNT_Other',
       'COMMUNITYRECOVERY', 'EVENT', 'GRIEF', 'GRIEF_Individual',
       'GRIEF_Community', 'GRIEF_Societal', 'HERO', 'INVESTIGATION', 'JOURNEY',
       'JOURNEY_Mental', 'JOURNEY_Physical', 'LEGAL', 'MEDIA', 'MISCELLANEOUS',
       'MOURNING', 'MOURNING_Individual', 'MOURNING_Community',
       'MOURNING_Societal', 'PERPETRATOR', 'PHOTO', 'POLICY', 'POLICY_Guns',
       'POLICY_InfoSharing', 'POLICY_MentalHealth', 'POLICY_Other',
       'POLICY_VictimAdv', 'POLICY_OtherAdv', 'POLICY_Practice',
       'PRIVATESECTOR', 'RACECULTURE', 'RESOURCES', 'SAFETY',
       'SAFETY_Community', 'SAFETY_Individual', 'SAFETY_SchoolOrg',
       'SAFETY_Societal', 'SOCIALSUPPORT', 'THREAT', 'THREAT_Assessment',
       'TRAUMA', 'TRAUMA_Physical', 'TRAUMA_Psychological',
       'TRAUMA_Individual', 'TRAUMA_Community', 'TRAUMA_Societal', 'VICTIMS',
       'word_

In [7]:
data = data[['StoryID', 'Excerpt', 'Excerpt_pre', 'Excerpt_tokenized',
       'Excerpt_nonstop', 'Excerpt_stemmed', 'Excerpt_lemmatized',
       'Excerpt_clean', 'ACCOUNT_Cultural']]

In [8]:
data.shape

(8131, 9)

In [9]:
data.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Cultural
0,NI3079,Elliot Rodger Wangs roommate stabbed Wang and ...,elliot rodger wangs roommate stabbed wang two ...,"['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...",0
1,NI2689,Because the tragedy unfolded at the start of a...,tragedy unfolded start holiday weekend school ...,"['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...",0
2,NI2400,Gunrelated restraining orders Also passed in r...,gunrelated restraining orders also passed resp...,"['gunrelated', 'restraining', 'orders', 'also'...","['gunrelated', 'restraining', 'orders', 'also'...","['gunrel', 'restrain', 'order', 'also', 'pass'...","['gunrelated', 'restraining', 'order', 'also',...","['gunrel', 'restrain', 'order', 'also', 'pass'...",0
3,NI1393,Linder said she will also remember Weiss for h...,linder said also remember weiss epic shot pool...,"['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'rememb', 'weiss', ...","['linder', 'said', 'also', 'remember', 'wei', ...","['linder', 'said', 'also', 'rememb', 'weiss', ...",0
4,NI3259,I mean I know how this goes We all do Werent y...,mean know goes werent sort expecting father on...,"['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...","['mean', 'know', 'go', 'werent', 'sort', 'expe...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...",1


In [11]:
data_uncl = data.loc[data['ACCOUNT_Cultural']==0]
data_uncl.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Cultural
0,NI3079,Elliot Rodger Wangs roommate stabbed Wang and ...,elliot rodger wangs roommate stabbed wang two ...,"['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...",0
1,NI2689,Because the tragedy unfolded at the start of a...,tragedy unfolded start holiday weekend school ...,"['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...",0
2,NI2400,Gunrelated restraining orders Also passed in r...,gunrelated restraining orders also passed resp...,"['gunrelated', 'restraining', 'orders', 'also'...","['gunrelated', 'restraining', 'orders', 'also'...","['gunrel', 'restrain', 'order', 'also', 'pass'...","['gunrelated', 'restraining', 'order', 'also',...","['gunrel', 'restrain', 'order', 'also', 'pass'...",0
3,NI1393,Linder said she will also remember Weiss for h...,linder said also remember weiss epic shot pool...,"['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'rememb', 'weiss', ...","['linder', 'said', 'also', 'remember', 'wei', ...","['linder', 'said', 'also', 'rememb', 'weiss', ...",0
5,NI1186,But three days after Cheng Yuan James Hong Ge...,three days cheng yuan james hong george chen w...,"['three', 'days', 'cheng', 'yuan', 'james', 'h...","['three', 'days', 'cheng', 'yuan', 'james', 'h...","['three', 'day', 'cheng', 'yuan', 'jame', 'hon...","['three', 'day', 'cheng', 'yuan', 'james', 'ho...","['three', 'day', 'cheng', 'yuan', 'jame', 'hon...",0


In [12]:
data_uncl.ACCOUNT_Cultural.unique()

array([0])

In [13]:
data_uncl.shape

(6978, 9)

In [14]:
data_cl = data.loc[data['ACCOUNT_Cultural']==1]
data_cl.shape

(1153, 9)

In [15]:
data_uncl = data_uncl.sample(1153)
data_uncl.shape

(1153, 9)

In [16]:
data_uncl.shape

(1153, 9)

In [17]:
data_cl.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Cultural
4,NI3259,I mean I know how this goes We all do Werent y...,mean know goes werent sort expecting father on...,"['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...","['mean', 'know', 'go', 'werent', 'sort', 'expe...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...",1
19,NI1111,At the earlier press conference Martinez shook...,earlier press conference martinez shook visibl...,"['earlier', 'press', 'conference', 'martinez',...","['earlier', 'press', 'conference', 'martinez',...","['earlier', 'press', 'confer', 'martinez', 'sh...","['earlier', 'press', 'conference', 'martinez',...","['earlier', 'press', 'confer', 'martinez', 'sh...",1
30,NI3229,Shall I observe that a gun is a weapon of mass...,shall observe gun weapon mass destruction ment...,"['shall', 'observe', 'gun', 'weapon', 'mass', ...","['shall', 'observe', 'gun', 'weapon', 'mass', ...","['shall', 'observ', 'gun', 'weapon', 'mass', '...","['shall', 'observe', 'gun', 'weapon', 'mass', ...","['shall', 'observ', 'gun', 'weapon', 'mass', '...",1
36,NI2075,At the earlier press conference Martinez shook...,earlier press conference martinez shook visibl...,"['earlier', 'press', 'conference', 'martinez',...","['earlier', 'press', 'conference', 'martinez',...","['earlier', 'press', 'confer', 'martinez', 'sh...","['earlier', 'press', 'conference', 'martinez',...","['earlier', 'press', 'confer', 'martinez', 'sh...",1
43,NI2719,Before the killings Rodgers mother became alar...,killings rodgers mother became alarmed bizarre...,"['killings', 'rodgers', 'mother', 'became', 'a...","['killings', 'rodgers', 'mother', 'became', 'a...","['kill', 'rodger', 'mother', 'becam', 'alarm',...","['killing', 'rodgers', 'mother', 'became', 'al...","['kill', 'rodger', 'mother', 'becam', 'alarm',...",1


In [18]:
data_cl.ACCOUNT_Cultural.unique()

array([1])

In [19]:
dataframe = [data_uncl, data_cl]
data = pd.concat(dataframe)
data.shape

(2306, 9)

In [21]:
data['Excerpt'].fillna("unknown", inplace=True)

In [23]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Cultural
7899,NI2612,Hackworth also touched on Coopers worldliness ...,hackworth also touched coopers worldliness sta...,"['hackworth', 'also', 'touched', 'coopers', 'w...","['hackworth', 'also', 'touched', 'coopers', 'w...","['hackworth', 'also', 'touch', 'cooper', 'worl...","['hackworth', 'also', 'touched', 'cooper', 'wo...","['hackworth', 'also', 'touch', 'cooper', 'worl...",0
2271,NI2941,Something is certainly rotten After the Sandy ...,something certainly rotten sandy hook massacre...,"['something', 'certainly', 'rotten', 'sandy', ...","['something', 'certainly', 'rotten', 'sandy', ...","['someth', 'certainli', 'rotten', 'sandi', 'ho...","['something', 'certainly', 'rotten', 'sandy', ...","['someth', 'certainli', 'rotten', 'sandi', 'ho...",1
2742,NI2586,The parents of the first three victims of Elli...,parents first three victims elliot rodgers mur...,"['parents', 'first', 'three', 'victims', 'elli...","['parents', 'first', 'three', 'victims', 'elli...","['parent', 'first', 'three', 'victim', 'elliot...","['parent', 'first', 'three', 'victim', 'elliot...","['parent', 'first', 'three', 'victim', 'elliot...",1
4313,NI2031,Police say Rodger killed six people stabbing ...,police say rodger killed six people stabbing t...,"['police', 'say', 'rodger', 'killed', 'six', '...","['police', 'say', 'rodger', 'killed', 'six', '...","['polic', 'say', 'rodger', 'kill', 'six', 'peo...","['police', 'say', 'rodger', 'killed', 'six', '...","['polic', 'say', 'rodger', 'kill', 'six', 'peo...",0
5043,NI2663,The relative shared other memories of Hong suc...,relative shared memories hong taking vegetaria...,"['relative', 'shared', 'memories', 'hong', 'ta...","['relative', 'shared', 'memories', 'hong', 'ta...","['rel', 'share', 'memori', 'hong', 'take', 've...","['relative', 'shared', 'memory', 'hong', 'taki...","['rel', 'share', 'memori', 'hong', 'take', 've...",0


In [24]:
data.ACCOUNT_Cultural.unique()

array([0, 1])

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306 entries, 7899 to 5849
Data columns (total 9 columns):
StoryID               2306 non-null object
Excerpt               2306 non-null object
Excerpt_pre           2306 non-null object
Excerpt_tokenized     2306 non-null object
Excerpt_nonstop       2306 non-null object
Excerpt_stemmed       2306 non-null object
Excerpt_lemmatized    2306 non-null object
Excerpt_clean         2306 non-null object
ACCOUNT_Cultural      2306 non-null int64
dtypes: int64(1), object(8)
memory usage: 180.2+ KB


## SPLITTING THE DATASET 

In [26]:
df = data['Excerpt']
df.shape

(2306,)

In [27]:
df.head()

7899    Hackworth also touched on Coopers worldliness ...
2271    Something is certainly rotten After the Sandy ...
2742    The parents of the first three victims of Elli...
4313    Police say Rodger killed six people  stabbing ...
5043    The relative shared other memories of Hong suc...
Name: Excerpt, dtype: object

In [28]:
y = data['ACCOUNT_Cultural']
y = y.to_numpy()
y

array([0, 1, 1, ..., 1, 1, 1])

In [29]:
type(y)

numpy.ndarray

In [30]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 306  
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(data, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((2000,), (2000,), (306,), (306,))

In [31]:
X_train.head()

7899    Hackworth also touched on Coopers worldliness ...
2271    Something is certainly rotten After the Sandy ...
2742    The parents of the first three victims of Elli...
4313    Police say Rodger killed six people  stabbing ...
5043    The relative shared other memories of Hong suc...
Name: Excerpt, dtype: object

In [32]:
X_valid.head()

2789    Santa Barbara mass shooting suspect Elliot Rod...
3756    Authorities do a wellbeing check on Rodger aft...
311     Nobody seems to care anymore that its easy for...
2286    But what remains unclear is how much police an...
7143    Conservative radio host Rush Limbaugh said Tue...
Name: Excerpt, dtype: object

In [33]:
y_valid

array([1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1])

In [34]:
y_train

array([0, 1, 1, ..., 1, 0, 0])

## MODEL DEPLOYMENT 

In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [36]:
import nltk
from nltk.tokenize import TweetTokenizer 
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

In [37]:
n = X_train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(X_train)
test_term_doc = vec.transform(X_valid)

In [38]:
trn_term_doc, test_term_doc

(<2000x18432 sparse matrix of type '<class 'numpy.float64'>'
 	with 260527 stored elements in Compressed Sparse Row format>,
 <306x18432 sparse matrix of type '<class 'numpy.float64'>'
 	with 36839 stored elements in Compressed Sparse Row format>)

In [39]:
trn_doc = trn_term_doc
val_doc = test_term_doc

In [40]:
trn_doc[0]

<1x18432 sparse matrix of type '<class 'numpy.float64'>'
	with 251 stored elements in Compressed Sparse Row format>

In [41]:
vocab = vec.get_feature_names()

In [42]:
vocab[5000:5010]

['else obviously',
 'else or',
 'else rodger',
 'else something',
 'else the',
 'elude',
 'elude them',
 'email',
 'email to',
 'emailed']

## NAIVE BAYES 

In [43]:
x = trn_doc
x

<2000x18432 sparse matrix of type '<class 'numpy.float64'>'
	with 260527 stored elements in Compressed Sparse Row format>

In [44]:
type(y_train)

numpy.ndarray

In [45]:
y = y_train
y.shape

(2000,)

In [46]:
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1

In [47]:
p

matrix([[1.     , 1.81463, 1.     , ..., 1.98691, 1.58288, 1.58288]])

In [48]:
p.shape

(1, 18432)

In [49]:
q

matrix([[2.4963 , 2.72104, 1.19179, ..., 1.     , 1.     , 1.     ]])

In [50]:
q.shape

(1, 18432)

In [51]:
r = np.log((p/p.sum())/(q/q.sum()))

In [52]:
r

matrix([[-0.98793, -0.47825, -0.24858, ...,  0.61346,  0.38613,  0.38613]])

In [53]:
r.shape

(1, 18432)

In [54]:
b = np.log(len(p)/len(q))

In [55]:
b

0.0

In [56]:
type(y_valid)

numpy.ndarray

In [57]:
y_valid.shape

(306,)

In [58]:
val_y = y_valid.reshape(1, 306)
val_y.shape

(1, 306)

In [59]:
pre_preds = val_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.9084967320261438

In [60]:
pre_preds = val_doc.sign() @ r.T + b
preds = pre_preds.T > 0
(preds==val_y).mean()

0.9084967320261438

## LOGISTIC REGRESSION 

In [61]:
x

<2000x18432 sparse matrix of type '<class 'numpy.float64'>'
	with 260527 stored elements in Compressed Sparse Row format>

In [62]:
y

array([0, 1, 1, ..., 1, 0, 0])

In [63]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(x, y)
preds = m.predict(val_doc)
(preds==val_y).mean()

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9117647058823529

In [64]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(trn_doc.sign(), y)
preds = m.predict(val_doc.sign())
(preds==val_y).mean()

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8921568627450981

In [65]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_doc, y)
preds = m.predict(val_doc)
(preds==val_y).mean()

0.9052287581699346

In [66]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_doc.sign(), y)
preds = m.predict(val_doc.sign())
(preds==val_y).mean()

0.9117647058823529

## TRI-GRAM MODEL DEPLOYMENT 

In [67]:
n = X_train.shape[0]
vect = TfidfVectorizer(ngram_range=(1,4), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vect.fit_transform(X_train)
test_term_doc = vect.transform(X_valid)

In [68]:
trn_doc = trn_term_doc

In [69]:
val_doc = test_term_doc

In [70]:
trn_doc.shape

(2000, 49392)

In [71]:
vocab = vect.get_feature_names()

In [72]:
vocab[20000:20005]

['identity of the person',
 'ideology',
 'ideology his',
 'ideology his horrific',
 'ideology his horrific sick']

In [73]:
y = y_train
y

array([0, 1, 1, ..., 1, 0, 0])

In [74]:
val_y

array([[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
        0, 1, 1, 0, 1, 1, 1, 1, 1]])

In [75]:
x = trn_doc.sign()
x

<2000x49392 sparse matrix of type '<class 'numpy.float64'>'
	with 452484 stored elements in Compressed Sparse Row format>

In [76]:
val_x = val_doc.sign()
val_x

<306x49392 sparse matrix of type '<class 'numpy.float64'>'
	with 59868 stored elements in Compressed Sparse Row format>

In [77]:
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [78]:
pre_preds = val_x @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.8758169934640523

In [79]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x,y)
preds = m.predict(val_x)
(preds==val_y).mean()

0.9150326797385621

In [80]:
r.shape

(1, 49392)

In [81]:
r

matrix([[-2.41306, -1.04697, -1.85345, ...,  1.61229,  1.61229,  1.61229]])

In [82]:
np.exp(r)

matrix([[0.08954, 0.351  , 0.1567 , ..., 5.01427, 5.01427, 5.01427]])

In [83]:
x_nb = x.multiply(r)
m = LogisticRegression(C=0.1, dual=True)
m.fit(x_nb,y)

val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
(preds==val_y).mean()

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9150326797385621